<h3> imports <h3>

In [6]:
import os, yaml

import time, threading, re, random
from collections import defaultdict
import urllib.parse as urlparse
import requests
from urllib.robotparser import RobotFileParser
import tldextract

from bs4 import BeautifulSoup
from datetime import datetime
import urllib.parse as urlparse
import pandas as pd




c:\Users\niush\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\niush\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
c:\Users\niush\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
C:\Users\niush\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


<h1> 1.	Crawler / Scraper <h1>

Defining basic settings

In [2]:
os.makedirs("configs", exist_ok=True)
os.makedirs("storage/raw", exist_ok=True)
os.makedirs("storage/clean", exist_ok=True)
os.makedirs("logs", exist_ok=True)

CONFIG = {
    "user_agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36",
    "default_timeout_sec": 15,
    "per_host_min_delay_sec": 2,
    "max_retries": 3,
    "sources": {
        "reddit": {
            "type": "reddit_html",
            "subreddits": ["worldnews", "news"],
            "limit_per_sub": 20
        },
        "news_sites": [
            {
                "name": "reuters_world",
                "start_urls": ["https://www.reuters.com/world/"],
                "article_selector_hint": "a"
            }
        ]
    }
}

with open("configs/crawl_config.yaml", "w", encoding="utf-8") as f:
    yaml.safe_dump(CONFIG, f, allow_unicode=True)
print("configs/crawl_config.yaml saved")


configs/crawl_config.yaml saved


Review the robots.txt and control the rate

In [4]:
_session = requests.Session()
_session.headers.update({"Accept-Language": "en;q=0.9"})

_robot_cache = {}
_host_next_time = defaultdict(float)
_lock = threading.Lock()

def get_robots_parser(base_url, ua):
    parsed = urlparse.urlparse(base_url)
    robots_url = f"{parsed.scheme}://{parsed.netloc}/robots.txt"
    if robots_url in _robot_cache:
        return _robot_cache[robots_url]
    rp = RobotFileParser()
    try:
        resp = _session.get(robots_url, timeout=8)
        if resp.status_code == 200:
            rp.parse(resp.text.splitlines())
            print("resp.status_code == 200")
        else:
            rp.parse([])
            print("resp.status_code != 200")
    except Exception:
        rp.parse([])
    _robot_cache[robots_url] = rp
    return rp

def host_key(url):
    p = urlparse.urlparse(url)
    ext = tldextract.extract(p.netloc)
    return ".".join([x for x in [ext.domain, ext.suffix] if x])

def rate_limit(url, min_delay_sec):
    hk = host_key(url)
    with _lock:
        now = time.time()
        nt = _host_next_time[hk]
        wait = nt - now
        if wait > 0:
            time.sleep(wait)
        _host_next_time[hk] = time.time() + min_delay_sec


Fetch function (with Retry & Exponential Backoff)

In [5]:
def fetch(url, user_agent, timeout=15, max_retries=3, min_delay_sec=2):
    rp = get_robots_parser(url, user_agent)
    if not rp.can_fetch(user_agent, url):
        raise PermissionError(f"Blocked by robots.txt for {url}")
    rate_limit(url, min_delay_sec)
    headers = {"User-Agent": user_agent}
    attempt = 0
    backoff = 1.6
    while attempt <= max_retries:
        try:
            resp = _session.get(url, headers=headers, timeout=timeout)
            if 200 <= resp.status_code < 300:
                return resp
            if resp.status_code in (429, 503):
                time.sleep((backoff ** attempt) + random.uniform(0, 0.5))
            else:
                time.sleep(0.6)
        except requests.RequestException:
            time.sleep((backoff ** attempt) + 0.4)
        attempt += 1
    raise TimeoutError(f"Fetch failed after retries for {url}")


HTML Crawler for Reddit

In [7]:
def parse_reddit_listing(html, base_url):
    soup = BeautifulSoup(html, "lxml")
    items = []
    # Generic selector for post container
    for post in soup.select("[data-testid='post-container']"):
        title_el = post.select_one("h3")
        if not title_el:
            continue
        title = title_el.get_text(strip=True)
        link_el = post.find("a", href=True)
        url = urlparse.urljoin(base_url, link_el["href"]) if link_el else None

        items.append({
            "source_type": "reddit",
            "source_name": base_url,
            "subreddit": base_url.rstrip("/").split("/")[-1],
            "url": url,
            "canonical_url": url,
            "title": title,
            "text": None,
            "author": None,
            "published_at": None,
            "score": None,
            "comments": None,
            "fetched_at": datetime.utcnow().isoformat(timespec="seconds")
        })
    return items

def crawl_reddit_subreddit(sub, cfg):
    ua = cfg["user_agent"]
    limit = cfg["sources"]["reddit"]["limit_per_sub"]
    url = f"https://www.reddit.com/r/{sub}/"
    resp = fetch(
        url, ua, timeout=cfg["default_timeout_sec"], 
        max_retries=cfg["max_retries"], 
        min_delay_sec=cfg["per_host_min_delay_sec"]
    )
    items = parse_reddit_listing(resp.text, url)
    return items[:limit] if limit else items
